In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
recipes = pd.read_csv("drive/MyDrive/Fall 2024: Study Abroad/Recipes Generator/Data/recipes_data.csv")

In [3]:
# Library imports

import numpy as np
import keras
from keras import Input, Model
from keras.layers import LSTM, Dense

In [4]:
# Preprocess the data

recipes = recipes[:10000]
recipes = recipes[recipes["NER"].str.len() <= 100]

ingredients_raw = []
names_raw = []
ingrs_chars = set()
names_chars = set()

for (ingredients, name) in zip(recipes["NER"], recipes["title"]):
  ingredients_raw.append(''.join(ingredients))
  name = '\t' + name + '\n'
  names_raw.append(name)

  for char in ingredients:
    if char not in ingrs_chars:
        ingrs_chars.add(char)
  for char in name:
    if char not in names_chars:
        names_chars.add(char)

ingrs_chars = sorted(list(ingrs_chars))
names_chars = sorted(list(names_chars))
num_encoder_tokens = len(ingrs_chars)
num_decoder_tokens = len(names_chars)
max_encoder_seq_length = max([len(txt) for txt in ingredients_raw])
max_decoder_seq_length = max([len(txt) for txt in names_raw])

print("Number of recipes:", len(ingredients_raw))
print("Number of unique ingredient tokens:", num_encoder_tokens)
print("Number of unique name tokens:", num_decoder_tokens)
print("Max sequence length for ingredients:", max_encoder_seq_length)
print("Max sequence length for names:", max_decoder_seq_length)

Number of recipes: 6307
Number of unique ingredient tokens: 39
Number of unique name tokens: 78
Max sequence length for ingredients: 100
Max sequence length for names: 88


In [5]:
# Tokenize the data

ingrs_token_index = dict([(char, i) for i, char in enumerate(ingrs_chars)])
names_token_index = dict([(char, i) for i, char in enumerate(names_chars)])

encoder_input_data = np.zeros(
    (len(ingredients_raw), max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
)
decoder_input_data = np.zeros(
    (len(ingredients_raw), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)
decoder_target_data = np.zeros(
    (len(ingredients_raw), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)

for i, (ingredients, name) in enumerate(zip(ingredients_raw, names_raw)):
    for t, char in enumerate(ingredients):
        encoder_input_data[i, t, ingrs_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, ingrs_token_index[" "]] = 1.0

    for t, char in enumerate(name):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, names_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, names_token_index[char]] = 1.0

    decoder_input_data[i, t + 1 :, names_token_index[" "]] = 1.0
    decoder_target_data[i, t:, names_token_index[" "]] = 1.0

In [6]:
latent_dim = 256
batch_size = 64
epochs = 200

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [7]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s_model.keras")

Epoch 1/200
79/79 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.7519 - loss: 1.4321 - val_accuracy: 0.8068 - val_loss: 0.8865
Epoch 2/200
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.8045 - loss: 0.8133 - val_accuracy: 0.8056 - val_loss: 0.7643
Epoch 3/200
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.8070 - loss: 0.7651 - val_accuracy: 0.8096 - val_loss: 0.7062
Epoch 4/200
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.8094 - loss: 0.7089 - val_accuracy: 0.8102 - val_loss: 0.6766
Epoch 5/200
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.8123 - loss: 0.6810 - val_accuracy: 0.8149 - val_loss: 0.6471
Epoch 6/200
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.8207 - loss: 0.6329 - val_accuracy: 0.8307 - val_loss: 0.5929
Epoch 7/200
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8302 - loss: 0.5938 - val_accuracy: 0.8309 - val_loss: 0.5738
Epoch 8/200
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.8377 - loss: 0.5609 - val_accuracy: 0.

In [8]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s_model.keras")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in ingrs_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in names_token_index.items())

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, names_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = sample(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [9]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", ingredients_raw[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: ["bite size shredded rice biscuits", "vanilla", "brown sugar", "nuts", "milk", "butter"]
Decoded sentence: Salmon Pasta Salad

-
Input sentence: ["cream of mushroom soup", "beef", "sour cream", "chicken breasts"]
Decoded sentence: Freahy Creamy Arpped Skry On Breaks

-
Input sentence: ["frozen corn", "pepper", "cream cheese", "garlic powder", "butter", "salt"]
Decoded sentence: Pickled Pepco Fite

-
Input sentence: ["chicken gravy", "cream of mushroom soup", "chicken", "shredded cheese"]
Decoded sentence: Ber Cheese Balls

-
Input sentence: ["graham cracker crumbs", "powdered sugar", "peanut butter", "chocolate chips", "butter"]
Decoded sentence: Broccoli Corn Bread

-
Input sentence: ["buttermilk", "egg", "sugar", "vanilla", "soda", "flour", "rhubarb", "butter", "salt"]
Decoded sentence: Sour Cream Pound Cake

-
Input sentence: ["egg", "pepper", "crackers", "cream-style corn", "whole kernel corn", "butter"]
Decoded sentence: Broccoli Dip

-
Input sentence: ["oil", "t